In [1]:
import symjax as sj
import symjax.tensor as T
import numpy as np

In [2]:
test = (2,2,2,2)
inp = tuple(list(test[4])) = test

SyntaxError: can't assign to function call (<ipython-input-2-774f2f1fef76>, line 2)

In [3]:
def body_func(inp, carry):
    print("in body function")
    tiprime_iter = inp[0]
    ti_iter = inp[1]
    # carry = T.index_update(carry, (1,1), 2)
    test = T.index_update(inp[4], (1,1), 2)
    inplist = list(inp)
    inplist[4] = test
    inp = tuple(inplist)
    # carry[ti_iter, tiprime_iter] = ti_iter 
    return (tiprime_iter + 1, ti_iter + 1, inp[2], inp[3], inp[4])

In [4]:
def cond_funct(inp):
    print("in cond function")
    tip_iter = inp[0]
    ti_iter = inp[1]
    dim1max = inp[2]
    dim2max = inp[3]
    # print(dim1max)
    # print(tip_iter)
    # return T.less_equal(tip_iter, dim1max)
    return T.logical_and(T.less_equal(tip_iter, dim1max), T.less_equal(ti_iter, dim2max))

In [5]:
w = T.Variable(1.0, dtype="float32")
v = T.Placeholder((), "float32")
out = T.while_loop(
    lambda i, u: i[0] + u < 5,
    lambda i: (i[0] + 1.0, i[0] ** 2),
    (w, 1.0),
    non_sequences_cond=[v]
)
f = sj.function(v, outputs=out)
print(f(0))
# 5, 16
print(f(2))
# [3, 4]

(array(5., dtype=float32), array(16., dtype=float32))
(array(3., dtype=float32), array(4., dtype=float32))


In [6]:
signal = T.Placeholder((2,), "float32", name="signal")
dim1ph = T.Variable(2, "float32", "alpha")
dim2ph = T.Variable(2, "float32", "alpha")
ct = T.Placeholder((2,2), "float32", name = "diag_vals")
# ct = -np.ones(np.min([2,2]))
# nph = T.Variable(n, "float32", "alpha")
output = T.while_loop(
    lambda a: cond_funct(a), # condition
    lambda a,b: body_func(a,b), # body
    [signal[0], signal[1], dim1ph, dim2ph, ct], # init 
    non_sequences_body=[ct]
)
f = sj.function(signal, ct, outputs=output)

in cond function
in body function


In [7]:
c_test = np.zeros((2,2))
final_state = f(np.array([0,0]), c_test)

in cond function
in body function


In [8]:
c_test

array([[0., 0.],
       [0., 0.]])

In [9]:
final_state

(array(3, dtype=int32),
 array(3, dtype=int32),
 array(2, dtype=int32),
 array(2, dtype=int32),
 array([[0., 0.],
        [0., 2.]], dtype=float32))